In [18]:
import pandas as pd
import os
from fpdf import FPDF
from datetime import datetime
heute = datetime.today().strftime("%d.%m.%Y")

def frage_pauschal(name, prozentsatz, grundhonorar, daten_dict):
    pass  # Placeholder for implementation

def fragebogen():
    leistungsphasen = {}  # define empty or realistic dictionary if needed
    leistungskosten = {}
    try:
        leistungskosten["Fahrtkosten"] = round(float(input("Wie viel kostet die Fahrt pro km (default 0.4EUR) ? ") or 0.4), 2)
        leistungskosten["Stundensatz"] = round(float(input("Wie hoch ist der Stundensatz (default 95EUR) ? ") or 95), 2)
    except:
        leistungskosten = {
            "Fahrtkosten": 0.4,
            "Stundensatz": 95,
        }

    print("=== Fragebogen zur Angebotserstellung ===\n")
    daten_projekt, daten_bauherr, daten_honorar, daten_leistungen, daten = {}, {}, {}, {}, {}

    print("=== Information über das Projekt ===\n")
    daten_projekt["Firma/Name"] = input("Firma / Name: ")
    daten_projekt["Ansprechpartner"] = input("Ansprechpartner: ")
    daten_projekt["Adresse"] = input("Adresse des Projekts: Str. Nr., PLZ, Ort: ")
    daten_projekt["Telefonnummer"] = input("Telefonnummer: ")
    daten_projekt["E-Mail"] = input("E-Mail-Adresse: ")
    daten_projekt["Projektbezeichnung"] = input("Projektbezeichnung / Referenz: ")
    daten_projekt["Projekt-Nr"] = input("Projekt-Nummer: ")

    print("=== Information über den Bauherr ===\n")
    daten_bauherr["Bauherr"] = input("Name des Bauherr: ")
    daten_bauherr["Adresse Bauherr"] = input("Anschrift Bauherr: Str. Nr., PLZ, Ort: ")
    daten_bauherr["Kontakt"] = input("Kontaktdaten: Tel.Nr / E-Mail ")

    daten_honorar["Kosten"] = float(input("Gesamtkosten (anrechenbar): "))

    print("=== Projektleistungen / Teilhonorar ===\n")
    grundhonorar = daten_honorar["Kosten"]

    daten_leistungen["Fahrt"] = round(float(input("Fahrt [km]: ")), 2)
    daten_leistungen["Mitarbeiter Bestandsaufnahme"] = int(input("Wie viele Mitarbeiter waren vor Ort für die Bestandsaufnahme? "))
    daten_leistungen["Anzahl Autos"] = int(input("Anzahl der gefahrenen Autos: "))
    daten_leistungen["Fahrtkosten"] = leistungskosten["Fahrtkosten"] * daten_leistungen["Fahrt"] * daten_leistungen["Anzahl Autos"]
    daten_leistungen["Stunden vor Ort"] = round(float(input("Wie viele Stunden vor Ort (zB 3.5h)? ")), 2)
    daten_leistungen["Bestandsaufnahme"] = daten_leistungen["Mitarbeiter Bestandsaufnahme"] * daten_leistungen["Stunden vor Ort"] * leistungskosten["Stundensatz"]
    daten_leistungen["Stunden Auswertung"] = round(float(input("Wie viele Stunden für die Auswertung der Bestandsaufnahme (zB 3.5)? ")), 2)
    daten_leistungen["Mitarbeiter Auswertung"] = int(input("Wie viele Mitarbeiter haben bei der Auswertung der Bestandsaufnahme mitgewirkt? "))
    daten_leistungen["Auswertung Bestandsaufnahme"] = daten_leistungen["Mitarbeiter Auswertung"] * daten_leistungen["Stunden Auswertung"] * leistungskosten["Stundensatz"]
    daten_leistungen["Stunden Messkonzept"] = round(float(input("Wie viele Stunden für die Messkonzepterstellung (zB 4.5) ?")), 2)
    daten_leistungen["Mitarbeiter Messkonzept"] = int(input("Wie viele Mitarbeiter haben für die Erstellung des Messkonzepts gearbeitet? "))
    daten_leistungen["Messkonzepterstellung"] = daten_leistungen["Stunden Messkonzept"] * daten_leistungen["Mitarbeiter Messkonzept"] * leistungskosten["Stundensatz"]

    daten_leistungen["Gesamtkosten"] = daten_leistungen["Fahrtkosten"] + daten_leistungen["Messkonzepterstellung"] + daten_leistungen["Auswertung Bestandsaufnahme"]

    daten["Zwischensumme (netto)"] = daten_leistungen["Gesamtkosten"]
    daten["MwSt 19 %"] = round(daten["Zwischensumme (netto)"] * 0.19, 2)
    daten["Bruttohonorar"] = round(daten["Zwischensumme (netto)"] + daten["MwSt 19 %"], 2)

    leistungen = [
        "Fahrtkosten",
        "Bestandsaufnahme",
        "Auswertung Bestandsaufnahme",
        "Messkonzepterstellung"
    ]
    anzl = [
        daten_leistungen["Anzahl Autos"],
        daten_leistungen["Mitarbeiter Bestandsaufnahme"],
        daten_leistungen["Mitarbeiter Auswertung"],
        daten_leistungen["Mitarbeiter Messkonzept"]
    ]
    zeit = [
        daten_leistungen["Fahrt"],
        daten_leistungen["Stunden vor Ort"],
        daten_leistungen["Stunden Auswertung"],
        daten_leistungen["Stunden Messkonzept"]
    ]

    class FragebogenPDF(FPDF):
        def header(self):
            logo_width = 40
            x_pos = self.w - self.r_margin - logo_width
            self.image("DBE_Bild.png", x=x_pos, y=10, w=logo_width)
            self.set_font("Arial", "B", 12)
            self.cell(0, 6, f"Anlage zum Angebot {daten_projekt['Projektbezeichnung']}", ln=True, align="L")
            self.cell(0, 6, "Honorarabrechnung", ln=True, align="L")
            self.line(self.l_margin, self.get_y(), self.w - self.r_margin, self.get_y())
            self.ln(2)
            self.set_font("Arial", "", 12)
            self.cell(60, 6, "Projekt: ", align="L")
            self.cell(70, 6, f"{daten_projekt['Projekt-Nr']}", ln=True, align="C")
            self.cell(60, 6, "Projekt Kurzbezeichnung: ", align="L")
            self.cell(70, 6, f"{daten_projekt['Projektbezeichnung']}", ln=True, align="C")
            self.cell(0, 6, f"{daten_projekt['Firma/Name']}", ln=True, align="C")
            self.cell(0, 6, f"{daten_projekt['Adresse']}", ln=True, align="C")
            self.ln(2)
            self.cell(60, 6, "Auftraggeber: ", align="L")
            self.cell(70, 6, f"{daten_bauherr['Bauherr']}", ln=True, align="C")
            self.cell(0, 6, f"{daten_bauherr['Adresse Bauherr']}", ln=True, align="C")
            self.cell(0, 6, f"{daten_bauherr['Kontakt']}", ln=True, align="C")
            self.cell(0, 6, "", ln=True)
            self.set_font("Arial", "B", 13)
            self.cell(0, 10, "Pauschalangebot zur technischen Erhebung", ln=True, align="C")
            self.ln(5)
            self.set_font("Arial", "", 12)
            self.cell(60, 6, "Kosten Einheiten: ", ln=True, align="L")
            self.cell(60, 6, "Ingenieur Leistungen ", align="L")
            self.cell(60, 6, f"{leistungskosten['Stundensatz']} EUR/h", ln=True, align="C")
            self.cell(60, 6, "Fahrtkosten ", align="L")
            self.cell(60, 6, f"{leistungskosten['Fahrtkosten']} EUR/km", ln=True, align="C")

        def tabelle_leistungen(self, daten_leistungen):
            self.set_font("Arial", "B", 11)
            self.cell(10, 8, "Nr.", border=1, align="C")
            self.cell(80, 8, "Leistung", border=1, align="C")
            self.cell(30, 8, "Anzahl", border=1, align="C")
            self.cell(30, 8, "Dauer/Strecke", border=1, align="C")
            self.cell(45, 8, "Betrag [EUR]", border=1, ln=True, align="C")
            self.set_font("Arial", "", 11)
            for nummer, key in enumerate(leistungen, start=1):
                betrag = f"{daten_leistungen[key]:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
                self.cell(10, 8, str(nummer), border=1, align="C")
                self.cell(80, 8, key, border=1)
                zahlen = "Auto/s" if nummer == 1 else "Pers."
                self.cell(30, 8, f"{str(anzl[nummer-1])} {zahlen}", border=1, align="C")
                einheit = "km" if nummer == 1 else "h"
                self.cell(30, 8, f"{zeit[nummer-1]} {einheit}", border=1, align="C")
                self.cell(45, 8, f"{betrag} EUR", border=1, ln=True, align="R")
            self.set_font("Arial", "B", 11)
            gesamt = daten_leistungen.get("Gesamtkosten", 0.0)
            gesamt_str = f"{gesamt:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
            self.cell(150, 8, "Gesamtkosten", border=1, align="R")
            self.cell(45, 8, f"{gesamt_str} EUR", border=1, ln=True, align="R")
            self.ln(4)

        def zusammenfassung(self, daten):
            self.set_font("Arial", "", 11)
            for key, value in daten.items():
                betrag = f"{value:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
                self.cell(100, 8, f"{key}", align="L")
                self.cell(0, 8, f"{betrag} EUR", ln=True, align="R")

        def unterschrift(self):
            self.ln(50)
            self.set_font("Arial", "", 9)
            self.cell(0, 6, "DBE - Digital Building Engineering GmbH", align="L")
            self.ln(20)
            self.cell(0, 6, "Tobias Hiekel", ln=True, align="L")
            self.cell(0, 6, "Geschäftsführer", ln=True, align="L")
            self.ln(5)
            self.cell(0, 6, f"Berlin, {heute}", ln=True, align="L")

    pdf = FragebogenPDF()
    pdf.add_page()
    pdf.tabelle_leistungen(daten_leistungen)
    pdf.zusammenfassung(daten)
    pdf.unterschrift()

    pdf_name = f"Pauschale_Angebotserstellung_{daten_projekt['Firma/Name'].replace(' ', '_')}.pdf"
    pdf.output(pdf_name)
    print(f"PDF-Datei gespeichert als: {pdf_name}")

if __name__ == "__main__":
    fragebogen()


Wie viel kostet die Fahrt pro km (default 0.4EUR) ?  
Wie hoch ist der Stundensatz (default 95EUR) ?  


=== Fragebogen zur Angebotserstellung ===

=== Information über das Projekt ===



Firma / Name:  Charite
Ansprechpartner:  Mustermann
Adresse des Projekts: Str. Nr., PLZ, Ort:  Hermannstr. 114, 12051, Berlin
Telefonnummer:  030300303
E-Mail-Adresse:  mustermann@charite.de
Projektbezeichnung / Referenz:  P0001
Projekt-Nummer:  0001


=== Information über den Bauherr ===



Name des Bauherr:  Bauherr Mustermeier
Anschrift Bauherr: Str. Nr., PLZ, Ort:  Hermannstr. 110, 12051, Berlin
Kontaktdaten: Tel.Nr / E-Mail  0303030 / mustermeier@bauherr.de
Gesamtkosten (anrechenbar):  10000


=== Projektleistungen / Teilhonorar ===



Fahrt [km]:  30
Wie viele Mitarbeiter waren vor Ort für die Bestandsaufnahme?  3
Anzahl der gefahrenen Autos:  1
Wie viele Stunden vor Ort (zB 3.5h)?  8
Wie viele Stunden für die Auswertung der Bestandsaufnahme (zB 3.5)?  24
Wie viele Mitarbeiter haben bei der Auswertung der Bestandsaufnahme mitgewirkt?  4
Wie viele Stunden für die Messkonzepterstellung (zB 4.5) ? 40
Wie viele Mitarbeiter haben für die Erstellung des Messkonzepts gearbeitet?  4


PDF-Datei gespeichert als: Pauschale_Angebotserstellung_Charite.pdf
